In [1]:
import gym
from gym import Env
from gym.spaces import Discrete
import numpy as np
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [2]:
# Define the custom environment
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(9)
        print("*****************", self.action_space)
        self.observation_space = Discrete(9)
        self.path = []  # Start with the initial state
        self.state = 0
        self.episode = 100
        self.reward_matrix = np.array([
            [-10, 10, 0, 0, 0, 0, 0, 0, 0],
            [-20, -10, -50, 0, 10, 0, 0, 0, 0],
            [0, 0, -10, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, -10, 0, 0, 0, 0, 0],
            [0, -20, 0, -50, -10, -50, 0, 10, 0],
            [0, 0, 0, 0, 0, -10, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, -10, -20, 0],
            [0, 0, 0, 0, -20, 0, 50, -10, 100],
            [0, 0, 0, 0, 0, 0, 0, 0, -10],
        ])
        
    def step(self, action):
        new_state_path = -1
        if self.reward_matrix[self.state, action] in [-50, 100]:
            newstate = action
            reward = self.reward_matrix[self.state, action]
            self.state = newstate
            done = True
            new_state_path =newstate
        else:
            if self.state != action:
                if self.reward_matrix[self.state, action] != 0:
                    newstate = action
                    reward = self.reward_matrix[self.state, action]
                    self.state = newstate
                    new_state_path =newstate
                else:
                    newstate = self.state
                    reward = self.reward_matrix[self.state, action]
                    self.state = newstate
                    new_state_path =newstate
            else:
                newstate = self.state
                reward = self.reward_matrix[self.state, action]
                self.state = newstate
                new_state_path =newstate
                
        self.episode -= 1
        if self.episode <= 0:
            done = True
        else:
            done = False
        info = {}
        self.path.append(new_state_path)
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        print("Path taken:", self.path)  # Print the path taken
        self.path = []  # Reset the path for the next episode
        self.state = 0
        self.episode = 100
        return self.state



In [3]:
# Register the environment with gym
from gym.envs.registration import register
register(
    id='ShowerEnv-v0',
    entry_point='__main__:ShowerEnv',
)

In [5]:
# Create the environment
env = DummyVecEnv([lambda: gym.make('ShowerEnv-v0')])

***************** Discrete(9)


/home/bidyut/.local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [6]:
episodes = 5
# states=[]

for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        # print("old state--->",state)
        # print("Action--->",action)
        n_state, reward, done, info= env.step(action)
        # print("New state--->",n_state)
        # print("reward== ",reward)
        score+=reward
        # states.append(state)
        # states.append(n_state)
        
    print('Episode:{} Score:{}'.format(episode, score))
    # print("States covered-->",states)

env.close()

/home/bidyut/.local/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/home/bidyut/.local/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
/home/bidyut/.local/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'int'>`
  logger.warn(
/home/bidyut/.local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/base_vec_env.py:234: UserWarning: You tried to c

Path taken: []


IndexError: invalid index to scalar variable.

In [ ]:
model = DQN("MlpPolicy", env, verbose=1)


In [ ]:
model.learn(total_timesteps=5)


In [ ]:
# Evaluate the trained agent
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward} +/- {std_reward}")

In [ ]:
env.close()

In [ ]:
# Reset the environment to start a new episode
obs = env.reset()

# Initialize the path with the starting state
path = [obs]
score=0
# Run the episode
done = False
while not done:
    # Use the trained model to predict the next action
    action, _states = model.predict(obs)
    
    # Take the action in the environment
    obs, reward, done, info = env.step(action)
    score+=reward
    # Append the new state to the path
    path.append(obs)

print("Predicted path:", path)
print("cumulative reward",score)